## Импорт библиотек ##

In [ ]:
! pip install pyenchant

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 2.5 MB/s eta 0:00:00


In [ ]:
import nltk
import gdown
import string
import enchant 
import numpy as np
import pandas as pd
nltk.download('punkt')
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Импорт данных ##

### Импорт стоп слов "stop_words" ###

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1mdsG9-6bJNemv0vplEyAHZt-OnQCpYuu&export=download'
output = 'stop_words.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1mdsG9-6bJNemv0vplEyAHZt-OnQCpYuu&export=download
To: /content/stop_words.txt
100%|██████████| 9.75k/9.75k [00:00<00:00, 6.41MB/s]


'stop_words.txt'

In [ ]:
stop_words_list = pd.read_csv('/content/stop_words.txt', sep = ',')
stop_words_list = list(stop_words_list.columns)
stop_words_list.append("'s")

In [ ]:
stop_words = []
for word in stop_words_list:
  modif_word = ''
  for sym in word:
    if sym != ' ' and sym != '"':
      modif_word += sym
  stop_words.append(modif_word)

### Импорт данных с описанием "train_df" и "test_df" ###

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1vc6tvrdrHYVVUL_TBxsFlAFDusYylAkd&export=download'
output = 'train.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/u/0/uc?id=1UPQOQG3L47ydTi6myfEMdo2qLqYYf-NS&export=download'
output = 'test.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1vc6tvrdrHYVVUL_TBxsFlAFDusYylAkd&export=download
To: /content/train.txt
100%|██████████| 35.4M/35.4M [00:00<00:00, 100MB/s] 
Downloading...
From: https://drive.google.com/u/0/uc?id=1UPQOQG3L47ydTi6myfEMdo2qLqYYf-NS&export=download
To: /content/test.txt
100%|██████████| 35.4M/35.4M [00:02<00:00, 11.9MB/s]


'test.txt'

In [ ]:
train_df = pd.read_csv('/content/train.txt', sep = ':::')
train_df = pd.concat([pd.DataFrame([train_df.columns.values], columns=train_df.columns), train_df], ignore_index=True)
train_df.columns = ['index', 'name', 'genre', 'descriptions']
del train_df['index']

train_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,name,genre,descriptions
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [ ]:
test_df = pd.read_csv('/content/test.txt', sep = ':::')
test_df = pd.concat([pd.DataFrame([test_df.columns.values], columns=test_df.columns), test_df], ignore_index=True)
test_df.columns = ['index', 'name', 'genre', 'descriptions']
del test_df['index']

test_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,name,genre,descriptions
0,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
1,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
2,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
3,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
4,Er nu zhai (1955),drama,Before he was known internationally as a mart...


## Токенизация описания "tokenize_description" ##

In [ ]:
def tokenize_description(description : str):
  """ 
  Выделяет токены 
  Приводит к нижнему регистру
  Удаляет пунктуацию
  Удаляет стоп слова
  """
  tokens = nltk.word_tokenize(description)
  tokens = [i.lower() for i in tokens if i not in string.punctuation]
  tokens = [i for i in tokens if i not in stop_words]

  return tokens

In [ ]:
test_df['tokenize_description'] = test_df.descriptions.apply(tokenize_description)
train_df['tokenize_description'] = train_df.descriptions.apply(tokenize_description)

In [ ]:
test_df.head(2)

,name,genre,descriptions,tokenize_description
0,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar...","[l.r, brane, loves, life, car, apartment, job,..."
1,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch...","[spain, march, 1964, quico, naughty, child, be..."


In [ ]:
train_df.head(2)

,name,genre,descriptions,tokenize_description
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,"[listening, conversation, doctor, parents, 10-..."
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...,"[brother, sister, incestuous, relationship, cu..."


## Создание словаря на основе train_df "vacab_word_index" и "vacab_index_word" ##

In [ ]:
all_words = []
for description in train_df.tokenize_description:
  for word in description:
    all_words.append(word)
print('Кол-во всех токенов:', len(all_words))
all_words = np.array(all_words)
unique_words = np.unique(all_words, axis = 0)
# Удаляем все числовые токены
unique_words = np.array([word for word in unique_words if not word.isdigit()])
print('Кол-во уникальных токенов:', unique_words.shape[0])

Кол-во всех токенов: 2676360
Кол-во уникальных токенов: 146019


In [ ]:
# Фильтруем слова оставляя только буквы
words_only_letter = []
for word in unique_words:
  mod_word = ''
  for sym in word:
    if sym.isalpha():
      mod_word += sym
  words_only_letter.append(mod_word)

words_only_letter = [word for word in words_only_letter if word != '']

In [ ]:
# Фильтруем остовляя только граматически правильные слова
dictionary = enchant.Dict("en_US")
unique_words = [word for word in words_only_letter[13:] if dictionary.check(word)]
# Удаляем появифшиеся стоп слова
unique_words = np.array([word for word in unique_words if word not in stop_words])

In [ ]:
vacab_word_index = {unique_words[i] : i+3 for i in range(unique_words.shape[0])}
vacab_word_index["<PAD>"] = 0
vacab_word_index["<START>"] = 1
vacab_word_index["<UNKNOWN>"] = 2

vacab_index_word = dict([(value, key) for (key, value) in vacab_word_index.items()])

print('Длина словаря:', len(vacab_word_index))

Длина словаря: 40779


### Функции кодирования и декодирования текста ###

In [ ]:
def decode_review(text):
    return [vacab_index_word.get(i, vacab_index_word[2]) for i in text]

def encode_review(text):
    words = text.lower().split()
    words = ['<START>'] + words
    idxs = [vacab_word_index.get(word, vacab_word_index['<UNKNOWN>']) for word in words]
    return idxs

In [ ]:
print('The film is horrible i like it')
print(encode_review('The film is horrible i like it'))
print(decode_review(encode_review('The film is horrible i like it')))

The film is horrible i like it
[1, 2, 16752, 2, 20515, 2, 2, 2]
['<START>', '<UNKNOWN>', 'film', '<UNKNOWN>', 'horrible', '<UNKNOWN>', '<UNKNOWN>', '<UNKNOWN>']


## Создание тренеровачных и тестировачных данных "train_x", "train_y" и "test_x", "test_y" ##

In [ ]:
genre_dict = {test_df.genre.unique()[i] : i for i in range(test_df.genre.unique().shape[0])}

In [ ]:
def encode_description(tokens):
  new_descrip = ' '.join(token for token in tokens)
  text_index = encode_review(new_descrip)
  return text_index

In [ ]:
data_test = test_df[['genre', 'tokenize_description']]
data_train = train_df[['genre', 'tokenize_description']]

In [ ]:
data_test.head(3)

,genre,tokenize_description
0,thriller,"[l.r, brane, loves, life, car, apartment, job,..."
1,comedy,"[spain, march, 1964, quico, naughty, child, be..."
2,documentary,"[year, life, albin, family, shepherds, north, ..."


In [ ]:
data_train.head(3)

,genre,tokenize_description
0,drama,"[listening, conversation, doctor, parents, 10-..."
1,thriller,"[brother, sister, incestuous, relationship, cu..."
2,adult,"[bus, empties, students, field, trip, museum, ..."


In [ ]:
data_test['genre'] = [genre_dict[genre] for genre in data_test.genre]
data_train['genre'] = [genre_dict[genre] for genre in data_train.genre]

data_test['tokenize_description'] = data_test.tokenize_description.apply(encode_description)
data_train['tokenize_description'] = data_train.tokenize_description.apply(encode_description)

<ipython-input-24-eaa17f6105a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['genre'] = [genre_dict[genre] for genre in data_test.genre]
<ipython-input-24-eaa17f6105a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['genre'] = [genre_dict[genre] for genre in data_train.genre]
<ipython-input-24-eaa17f6105a2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
data_train.head()

,genre,tokenize_description
0,3,"[1, 24188, 10476, 13641, 29125, 2, 2, 23747, 1..."
1,0,"[1, 7044, 37124, 21466, 33734, 11455, 26838, 3..."
2,13,"[1, 7346, 14856, 39331, 16689, 41972, 26856, 2..."
3,3,"[1, 42793, 16408, 14977, 25479, 2, 42269, 3712..."
4,3,"[1, 16752, 41270, 33444, 2, 6404, 19063, 38737..."


In [ ]:
data_test.head()

,genre,tokenize_description
0,0,"[1, 2, 2, 24507, 24003, 7806, 4043, 22918, 184..."
1,1,"[1, 2, 25060, 2, 2, 27086, 8583, 5723, 44450, ..."
2,2,"[1, 45318, 24003, 2, 16278, 36597, 27587, 2, 1..."
3,3,"[1, 16408, 12835, 2, 38338, 7044, 2, 45330, 76..."
4,3,"[1, 22388, 25159, 4421, 39800, 2, 23775, 4106,..."


In [ ]:
# Максимальная длина описания
description_max_length = 10
for description in train_df.tokenize_description:
  if len(description) >= description_max_length:
    description_max_length = len(description)

In [ ]:
train_x = tf.keras.preprocessing.sequence.pad_sequences(
    data_train.tokenize_description,
    value=vacab_word_index["<PAD>"],
    padding='post',
    maxlen=description_max_length)

test_x = tf.keras.preprocessing.sequence.pad_sequences(
    data_test.tokenize_description,
    value=vacab_word_index["<PAD>"],
    padding='post',
    maxlen=description_max_length)

In [ ]:
train_y = data_train.genre
test_y = data_test.genre

In [ ]:
print('Максимальная длина описания:', description_max_length)
print('\nРазмер train_x:', train_x.shape)
print('Размер train_y:', train_y.shape)
print('\nРазмер test_x:', test_x.shape)
print('Размер test_y:', test_y.shape)

Максимальная длина описания: 1112

Размер train_x: (54214, 1112)
Размер train_y: (54214,)

Размер test_x: (54200, 1112)
Размер test_y: (54200,)


## Создание модели ##

In [ ]:
vector_size = 8
class_count = len(genre_dict)
batch_size = 512
epochs = 20
vocab_size = len(vacab_word_index)

In [ ]:
model = tf.keras.Sequential([
        Embedding(vocab_size, vector_size),
        Bidirectional(
        LSTM(vector_size, return_sequences=True)),
        Bidirectional(
        LSTM(vector_size, return_sequences=False)),
        Dense(128, activation = 'relu'),
        Dense(class_count, activation=tf.nn.softmax),
])

#model.compile(optimizer = Adam(learning_rate = 0.001), loss = SparseCategoricalCrossentropy(), metrics = ['accuracy'])

model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 8)           326232    
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 16)         1088      
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 16)               1600      
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               2176      
                                                                 
 dense_5 (Dense)             (None, 27)                3483      
                                                                 
Total params: 334,579
Trainable params: 334,579
Non-tr

In [ ]:
history = model.fit(train_x, train_y, batch_size = batch_size, epochs = epochs, validation_data = (test_x, test_y))

Необхадимо добавить механизм внимания для увеличения точности
https://analyticsindiamag.com/hands-on-guide-to-bi-lstm-with-attention/